# Segundo parcial - Algorítmica numérica

## Primera pate: **Preguntas teórico-prácticas** - sin apuntes, 60min, total 5 puntos

### Ejercicio 1
**[1.2 puntos] -** Los siguientes bloques de código evalúan operaciones sencillas. Sin embargo, los resultados que se obtienen no son intuitivos. Explica por qué se obtienen dichos resultados (**una/dos frases por item son suficientes**):


**(1.a)**

Al evaluar la siguiente integral por el método de Simpson con 101 subintervalos, el resultado no tiene nada que ver con el valor real de la integral:
$$
\int_{-5}^{5} \left| \frac{1}{x} \right| \, dx
$$

In [3]:
from scipy import integrate
import numpy as np

x = np.linspace(-5, 5, 102)  # 102 nodos -> 101 subintervalos
x = x[x != 0]  # Elimina el valor donde x = 0 para evitar división por cero
y = np.abs(1 / x)  # Cálculo de y evitando la división por cero

dx = x[1] - x[0]  # Tamaño de paso entre los valores de x
I = integrate.simpson(y, dx=dx)

print(I)  # Imprime el resultado aproximado de la integral


11.77096667272374


In [ ]:
from scipy import integrate

import numpy as np
x = np.linspace(-5, 5, 102)  # 102 nodos -> 101 subintervalos
y = np.array([abs(1/x_val) for x_val in x])
I = integrate.simpson(y, x)

print(I)  # 11.77096, pero el resultado exacto de la integral es infinito

**respuesta :** estamos hablando de un numero impar de subintervalos, por lo que tenemos que aplicar la regla 3/8 de simpson. Probablemente lo que este pasando es que de error calculando la primera parte con la formula de h/3 ... pero que se quede solo con el área de los 3 últimos subintervalos con el 3h/8....

-----------------------

**(1.b)**

El siguiente código implementa un simple algoritmo iterativo que converge rápidamente:

In [4]:
value = 1.0
max_ite = 1000
for i in range(max_ite):
    value += 1.0

    # Condición de parada
    if value == 9.0:
        print(f"Converged in {i} iterations")
        break
else:
    print("Failed to converge.")


Converged in 7 iterations


Observa ahora la siguiente modificación trivial del algoritmo anterior. Con esta modificación, el nuevo algoritmo no converge

In [5]:
value = 0.1  # CAMBIADO
max_ite = 1000
for i in range(max_ite):
    value += 0.1  # CAMBIADO

    # Condición de parada
    if value == 0.9:  # CAMBIADO
        print(f"Converged in {i} iterations")
        break
else:
    print("Failed to converge.")



Failed to converge.


**respuesta:** lo que sucede aqui es que 1 es representable en binario y 0.1 no tiene una representación exacta. por lo que el error que va arrastrando 0.1 al ir sumandose hace que no sea igual a 0.9

-----------------------

**(1c)**

Los mayores y menores números normales representables con precisión doble pueden obtenerse con el paquete ```sys``` usando ```sys.float_info.max``` y ```sys.float_info.min``` respectivamente. La multiplicación del número mayor por $100$ resulta en un desbordamiento (*overflow*):

In [9]:
import sys

print(100 * sys.float_info.max)  # overflow -> inf

inf



Sin embargo, la división del número menor por $100$ no produce un *underflow* y la operación es calculada correctamente:


In [10]:
import sys

print(sys.float_info.min / 100)  # 2.2250738585072e-310 -> resultado correcto

2.2250738585072e-310


**respuesta :** lo que sucede es que el maximo numero normal representable, ya esta usando toda la capacidad de la mantisa, mientras que el minimo normal representable no. or o que al dividir entre 100 lo que estamos haciendo es reducir la precision de la mantisa para aumentar la reresentación del exponente. por tanto `sys.float_info.min` es el minimo numero normal representable pero hay numeros subnormales más pequeños.

-----------------------

### Ejercicio 2

**[1 punto]** - ¿Cómo se llama el método que permite adaptar el procedimiento de interpolación de Newton basado en la tabla de diferencias divididas para casos donde se disponga de información sobre la primera derivada de la función en los nodos? Describe detalladamente este método de **una** de las siguientes tres formas:
 - Escribe el pseudocódigo del algoritmo en cuestión.
 - Implementa en Python dicho algoritmo.
 - Describe *de palabra* el procedimiento.

In [ ]:
# se llama interpolacion de hermite
# voy a implementar un pseudo codigo: lo voy a dividir en 2 funciones, una que calcule lso coeficientes y otra que evalue el polinomio y uan auxiliar que no me defino que es el factorial

def hermite_coefficients(x_data: list, y_data:list) -> list:
    '''
    aqui x_data contiene los datos de x
    y_data es uan lista anidada donde e primero elemnto sera la derivada 0, es decir los valores de y
    y el segundo elemento la primera derivada y asi sucesivamente
    '''

    feasible = len(todos los elemntos de y) = len(x)
    #feasible es un booleano que indica si faltan datos en alguno de las derivadas
    #si daltan datos se detiene la ejecucion con un valuerror
    if not feasible:
        raise ValueError(" coudn't use this method due to missing data")
    
    else:
        x_values = [x por cada eleemento en x_data tantas veces como len(y_data)] # pq len(y_data) es el nuemro de datos que tenemso de x
        y_values = [y por cada eleemento en y_data tantas veces como len(y_data)] 
        #almacenamos los coeficientes en uan lista vacia
        coefficients = []
        coefficients.append(y_values[0])
        layer = 1 # es la capa de diferencia finita en al que estamso calculando
        divided_diff = [] # para guardar los valores de las diferencias finitas y luego actualizar el y values

        # tener en cuenta si el xi = xi+1 para dividir entre el factorial del orden de la derivada o hacer al dif finita nroaml

        while len(y_values) > 1:
            for i in range(len(y_values) - 1):

                if x[i] == x[i + layer]:
                    value = y_values[1 // len(x_data)] / factorial(layer)

                else:
                    value = (y_values[i + 1] - y_values[i]) / (x_values[i + layer] - x_values[i])

                divided_diff.append(value)

            coefficients.append(divided_diff[0])
            y_values = divided_diff
            layer += 1
            divided_diff = [] #reseteamos la lista


def evaluate_hermite(coefficients, x_data, y_data, x_eval) -> float:

    herm_poly = coefficients[0] # el valor del polinomio
    product = 1 #para ir acumulando el producto y no calcularlo para cada valro de la lista
    x_values = [elemento for elemento in x_data for _ in range(len(y_data))] #repetios las x

    for i in range(1, len(coefficients)):
        product *= (x - x_values[i -1])

        value = coefficients[i] * product
        print('value = ', value, 'coef = ', coefficients[i], 'producto = ', product)
        herm_poly += value

        print(herm_poly)
    
    return herm_poly

    


-----------------------

### Ejercicio 3

**[1 punto] -**  Deduce razonadamente el **número de ecuaciones de cierre** que será necesario imponer para cerrar el problema del cálculo de un **spline de orden 4** (es decir, cada tramo será ajustado por un polinomio de grado 4). Las condiciones generales a imponer son las mismas que las usadas para el cálculo de un spline cúbico + la condición de continuidad de la derivada 3ª en los nodos internos. Denota por $n+1$ el número de nodos disponibles ($x_0, x_1, \dots , x_n$) y por $n$ el número de subintervalos, siendo $q_{i,i+1}(x)$ el polinomio que se extiende desde el nodo $i$ al $i+1$. 

**respuesta:** partimos de que un splin de orden 4 tendra uan forma del tipo ax^4 + bx^3 + cx^2 + dx + e, por tanto tenemos 5 incognitas. como tenemos n+1 puntos tendremos n intervalos. por lo que tenemos 5n ecuaciones. 

ahora sabemos que por cada punto deben pasar dos ecuaciones, la de entrada y la de salida por asi decirlo. tambien que cada ecuacion pasa por dos puntos, el origen y el final del intervalo. eso nos deja 2n ecuaciones. 


imponemos ahora que q´i-1,i(x) = q´i,i+1(x) de donde salen n-1 ecuaciones, 
imponemos esto de igaul manera para q´´ y q´´´ de donde salen n-1 ecuaciones para cada uan


esto nos queda 2n + (n-1) + (n-1) + (n-1) = 2n + 3(n-1) = 2n + 3n - 3 = 5n - 3

se lo restamos a nuestro numero inicial de ecuaciones 5n - (5n - 3) = 3 ecuaciones de cierre

-----------------------

### Ejercicio 4

**[0.8 punto] -** La regresión multilineal sigue la misma idea que la regresión lineal univariante. Observa el siguiente caso: en un experimento se han obtenido los siguientes datos de una determinada función real $z(x,y)$:

 <center>

|  x  |  y  |  z  |
|-----|-----|-----|
|  0  |  0  |  6  |
|  0  |  0  | -2  |
|  0  |  1  |  0  |
| -1  |  0  |  0  |

 </center>

Observa que ahora $x$ e $y$ son las variables independientes. Se desea obtener un modelo de la forma $\hat{z} = a + bx + cy$ que ajuste dichos puntos. Minimiza el error cuadrático medio (igual que harías con una única variable independiente) para demostrar que los coeficientes del modelo deben ser $a=2$, $b=2$ y $c=-2$.

In [13]:
import numpy as np

# Matriz de variables independientes, incluyendo el término constante para 'a'
X = np.array([
    [1, 0, 0],  # 1 para 'a', 0 para 'x', 0 para 'y'
    [1, 0, 0],
    [1, 0, 1],
    [1, -1, 0]
])

# Vector de valores dependientes 'z'
Z = np.array([6, -2, 0, 0])

# Cálculo de los coeficientes mediante mínimos cuadrados
coefficients = np.linalg.lstsq(X, Z)[0]
nose = np.linalg.lstsq(X, Z)
a, b, c = coefficients
print(f"a = {a}, b = {b}, c = {c}")
print(nose)


a = 2.0, b = 2.0000000000000004, c = -1.9999999999999993
(array([ 2.,  2., -2.]), array([32.]), np.int32(3), array([2.13577921, 1.        , 0.66215345]))


-----------------------

### Ejercicio 5

**[1 punto] -** ¿Qué relación existe entre la regresión de Ridge y el fenómeno del *overfit* (sobre-ajuste)? ¿Qué magnitud se minimiza realmente en el cálculo de la regresión lineal de Ridge? Discute si es verdadera o falsa la siguiente afirmación: *\"Un modelo basado en la regresión de Ridge tendrá una mejor calidad de ajuste sobre el conjunto de datos de entrenamiento (es decir, un mayor coeficiente $R_{train}^2$) en comparación con el mismo modelo sin penalización de Ridge\"*.

**respuesta :** la relacion que existe entre la regresion de ridge y el overfitting es que si tenemos un polinomio que interpola los datos o los ajusta mucho a los conocidos pero mal a los nuevos (overfit) la regresion de ridge lo que va a bsucar es aplanar ese polinomio de forma que sea mas general y prediga mejor datos nuevos. 

en la regrsion lineal de ridge la magnitud que se minimiza es el coeficiente de al x en uan recat de la forma r = a + bx, es decir que se minima b al multiplicarlo por un landa arbitrario. siempre que hagamos uan regresion linal, al aplicarle el coeficiente de ridge nos debe salir un valor de b menor a la recta que hemso predicho apra nuestro ajuste


la frase no es necesariamente cierta. un modelo con regresion de ridge será menos oscilatorio y más plano pero eso no quiere decir que sea mejor. en una funcion socilatoria como puede ser el coseno, un polinomio de interpolacion probablemente sea mejor qe  un modelo con la penalizacion de ridge.